In [1]:
import torch
import torchaudio
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from ecapa_model import ECAPA_TDNN  # Dosya yanındaysa bunu kullan

# Eğer import hatası alırsan yukarıdaki satırı silip ECAPA_TDNN class'ını buraya yapıştırabilirsin.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"İşlem yapılan cihaz: {device}")

C:\Users\omera\AppData\Roaming\Python\Python311\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


İşlem yapılan cihaz: cuda


In [3]:
def load_trained_model(model_path, C=256, emb_size=128):
    # Modeli başlat (Backbone)
    model = ECAPA_TDNN(C=C).to(device)
    
    print(f"Model yükleniyor: {model_path}")
    
    # Checkpoint'i yükle
    checkpoint = torch.load(model_path, map_location=device)
    
    # State Dict temizliği (speaker_encoder. prefixini atıyoruz)
    model_state = model.state_dict()
    new_state = {}
    
    for k, v in checkpoint.items():
        # Sadece encoder parametrelerini al
        if k.startswith("speaker_encoder."):
            name = k.replace("speaker_encoder.", "")
            new_state[name] = v
            
    # Parametreleri yükle
    model.load_state_dict(new_state)
    model.eval() # Inference moduna al
    print("Model başarıyla yüklendi ve hazır!")
    return model

# 22. Epoch modelini buraya tanımla
MODEL_PATH = r"C:\Users\omera\Desktop\youtube_data\models\model_0023.model" # Veya senin kaydettiğin tam yol
model = load_trained_model(MODEL_PATH, C=256)

Model yükleniyor: C:\Users\omera\Desktop\youtube_data\models\model_0023.model
Model başarıyla yüklendi ve hazır!


In [4]:
def get_embedding(model, audio_path):
    if not os.path.exists(audio_path):
        raise FileNotFoundError(f"Dosya bulunamadı: {audio_path}")
        
    # 1. Sesi Yükle
    wav, sr = torchaudio.load(audio_path)
    
    # 2. Resample (Eğer 24k değilse)
    if sr != 24000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=24000)
        wav = resampler(wav)
        
    # 3. Stereo ise Mono yap
    if wav.shape[0] > 1:
        wav = torch.mean(wav, dim=0, keepdim=True)
        
    # 4. Modele Ver (Batch boyutu ekle: [1, samples])
    wav = wav.to(device)
    
    with torch.no_grad():
        # aug=False önemli! Testte gürültü eklemiyoruz.
        embedding = model(wav, aug=False)
        
    return embedding

def compare_files(file1, file2):
    emb1 = get_embedding(model, file1)
    emb2 = get_embedding(model, file2)
    
    # Cosine Similarity
    similarity = nn.CosineSimilarity(dim=-1, eps=1e-6)
    score = similarity(emb1, emb2).item()
    
    return score

def interpret_score(score):
    print(f"Benzerlik Skoru: {score:.4f}")
    if score > 0.45:
        print("✅ SONUÇ: KESİNLİKLE AYNI KİŞİ")
    elif score > 0.35:
        print("🤔 SONUÇ: MUHTEMELEN AYNI KİŞİ (Sınırda)")
    else:
        print("❌ SONUÇ: FARKLI KİŞİ")

In [8]:
my_voice_1 = r"C:\Users\omera\Desktop\youtube_data\speaker_verification\omer_deneme.wav"      # Senin yüklediğin 1. kayıt
my_voice_2 = r"C:\Users\omera\Desktop\youtube_data\speaker_verification\omer_deneme_2.wav"    # Senin yüklediğin 2. kayıt

# Datasetten rastgele bir başkası (CSV'den veya klasörden bakıp yolunu yaz)
# Örnek: "dataset/baska_bir_konusmaci/ses_dosyasi.wav"
other_voice = r"C:\Users\omera\Desktop\youtube_data\speaker_verification\baskasi.wav" # Burayı kendi datasetine göre düzenle!

print("--- TEST 1: Kendi Sesin vs Kendi Sesin ---")
if os.path.exists(my_voice_1) and os.path.exists(my_voice_2):
    score = compare_files(my_voice_1, my_voice_2)
    interpret_score(score)
else:
    print("Lütfen ses dosyalarını yüklediğinden emin ol!")

print("\n--- TEST 2: Kendi Sesin vs Başkası ---")
if os.path.exists(my_voice_1) and os.path.exists(other_voice):
    score = compare_files(my_voice_1, other_voice)
    interpret_score(score)
else:
    print("Dataset dosya yolunu kontrol et!")

--- TEST 1: Kendi Sesin vs Kendi Sesin ---


C:\Users\omera\AppData\Roaming\Python\Python311\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


Benzerlik Skoru: 0.8934
✅ SONUÇ: KESİNLİKLE AYNI KİŞİ

--- TEST 2: Kendi Sesin vs Başkası ---
Benzerlik Skoru: 0.2042
❌ SONUÇ: FARKLI KİŞİ


In [5]:
import sounddevice as sd
import time
import soundfile as sf
SAMPLE_RATE = 24000  # Modelin eğitimiyle aynı olmalı!
DURATION = 4         # Kaç saniye kayıt alacağı
CHANNELS = 1         # Mono kayıt

def record_audio(filename, duration=DURATION, fs=SAMPLE_RATE):
    print(f"\n🎤 KAYIT BAŞLIYOR ({duration} saniye)...")
    print(f"Lütfen konuşun: '{filename}' için kayıt alınıyor.")
    time.sleep(0.5) # Hazırlık için kısa bekleme
    
    # Kayıt işlemi
    my_recording = sd.rec(int(duration * fs), samplerate=fs, channels=CHANNELS)
    sd.wait()  # Kaydın bitmesini bekle
    
    print("✅ Kayıt tamamlandı.")
    # Dosyayı kaydet
    sf.write(filename, my_recording, fs)
    print(f"Dosya kaydedildi: {filename}")

In [6]:
record_audio("omer_deneme.wav")


🎤 KAYIT BAŞLIYOR (4 saniye)...
Lütfen konuşun: 'omer_deneme.wav' için kayıt alınıyor.
✅ Kayıt tamamlandı.
Dosya kaydedildi: omer_deneme.wav


In [7]:
record_audio("omer_deneme_2.wav")


🎤 KAYIT BAŞLIYOR (4 saniye)...
Lütfen konuşun: 'omer_deneme_2.wav' için kayıt alınıyor.
✅ Kayıt tamamlandı.
Dosya kaydedildi: omer_deneme_2.wav


In [2]:
import pandas as pd
import random
import os
from itertools import combinations

# --- AYARLAR ---
VAL_CSV_PATH = r"C:\Users\omera\Desktop\speaker_verification\artifacts\val_split.csv"  # Senin ayırdığın validasyon dosyası
ROOT_PATH = r"C:\Users\omera\Desktop\youtube_data\dataset"                   # Dosyaların olduğu ana klasör
OUTPUT_FILE = "test_pairs_static.csv"   # Çıktı dosyası
NUM_PAIRS = 10000                       # Toplam çift sayısı (5000 Pozitif + 5000 Negatif)
SEED = 42                               # Sabit sonuç için random seed

def generate_static_pairs():
    random.seed(SEED)
    print(f"CSV Okunuyor: {VAL_CSV_PATH}")
    
    # CSV'yi oku
    df = pd.read_csv(VAL_CSV_PATH,sep="|")
    df.columns = [c.strip() for c in df.columns]
    
    # Kolon isimlerini belirle
    spk_col = 'speaker' if 'speaker' in df.columns else df.columns[2]
    aud_col = 'audio_path' if 'audio_path' in df.columns else df.columns[0]
    
    # Speaker -> Dosya Listesi haritası çıkar
    spk_to_files = {}
    for _, row in df.iterrows():
        spk = row[spk_col]
        fname = row[aud_col]
        full_path = os.path.join(ROOT_PATH, str(spk), str(fname))
        
        # Dosya var mı kontrol et (Gereksiz hata almamak için)
        if os.path.exists(full_path):
            if spk not in spk_to_files:
                spk_to_files[spk] = []
            spk_to_files[spk].append(full_path)

    # En az 2 dosyası olanlar (Pozitif çift için)
    valid_speakers = [s for s in spk_to_files if len(spk_to_files[s]) >= 2]
    all_speakers = list(spk_to_files.keys())
    
    print(f"Uygun Konuşmacı Sayısı: {len(valid_speakers)}")
    
    pairs = []
    
    # 1. Pozitif Çiftler (Aynı Kişi)
    print("Pozitif çiftler üretiliyor...")
    target_pos = NUM_PAIRS // 2
    for _ in range(target_pos):
        spk = random.choice(valid_speakers)
        f1, f2 = random.sample(spk_to_files[spk], 2)
        pairs.append([f1, f2, 1]) # 1 = Aynı

    # 2. Negatif Çiftler (Farklı Kişi)
    print("Negatif çiftler üretiliyor...")
    target_neg = NUM_PAIRS // 2
    for _ in range(target_neg):
        spk1, spk2 = random.sample(all_speakers, 2)
        while spk1 == spk2: # Farklı olmasını garanti et
            spk2 = random.choice(all_speakers)
            
        f1 = random.choice(spk_to_files[spk1])
        f2 = random.choice(spk_to_files[spk2])
        pairs.append([f1, f2, 0]) # 0 = Farklı
        
    # Karıştır ve Kaydet
    random.shuffle(pairs)
    
    # DataFrame olarak kaydet
    out_df = pd.DataFrame(pairs, columns=['file1', 'file2', 'label'])
    out_df.to_csv(OUTPUT_FILE, index=False)
    
    print(f"✅ Başarılı! {len(pairs)} çift '{OUTPUT_FILE}' dosyasına kaydedildi.")

if __name__ == "__main__":
    generate_static_pairs()

CSV Okunuyor: C:\Users\omera\Desktop\speaker_verification\artifacts\val_split.csv
Uygun Konuşmacı Sayısı: 2143
Pozitif çiftler üretiliyor...
Negatif çiftler üretiliyor...
✅ Başarılı! 10000 çift 'test_pairs_static.csv' dosyasına kaydedildi.
